## Maximun Marginal Retriver


In [1]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [4]:
vector_store = FAISS.from_documents(
	docs, 
	embedding=GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
)

In [15]:
retriver = vector_store.as_retriever(
	search_kwargs={"k": 3, 'lambda_mul': 0.5}, # lambda_mult = relevance-diversity balance ---> 1 : work as normal similarity search,  0 : work as MMR
	search_type="mmr"
)

In [16]:
query = "What is langchain?"
res = retriver.invoke(query)

In [17]:
for i, doc in enumerate(res):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain is used to build LLM based applications.

--- Result 2 ---
Embeddings are vector representations of text.

--- Result 3 ---
LangChain supports Chroma, FAISS, Pinecone, and more.


## Multi Query Retriver


In [ ]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_core.documents import Document
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [5]:
vector_store = FAISS.from_documents(
	all_docs, 
	embedding=GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
)


In [12]:
similarity_retriver = vector_store.as_retriever(
	search_kwargs={"k": 5}, 
	search_type="similarity"
)

In [13]:
mqr = MultiQueryRetriever.from_llm(
	llm=ChatGoogleGenerativeAI(model="gemini-2.5-pro"),
	retriever=vector_store.as_retriever(search_kwargs={"k": 5}),
)

In [14]:
# Query
query = "How to improve energy levels and maintain balance?"

In [15]:
similarity_result = similarity_retriver.invoke(query)
multi_query_result = mqr.invoke(query)

In [16]:
for i, doc in enumerate(similarity_result):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multi_query_result):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
The solar energy system in modern homes helps balance electricity demand.

--- Result 4 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 5 ---
Deep sleep is crucial for cellular repair and emotional regulation.
******************************************************************************************************************************************************

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Deep sleep is crucial for cellular repair and emotional regulation.

--- Result 4 ---
Regular walking boosts heart health and can reduce symptoms of depression

## ContexualCompression Retriver


In [1]:
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_classic.retrievers.document_compressors import LLMChainExtractor
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [3]:
embedding_model = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001")
vector_store = FAISS.from_documents(
	docs,
	embedding=embedding_model
)

In [4]:
base_retriver = vector_store.as_retriever(
	search_kwargs={"k": 5}
)

In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
compressor = LLMChainExtractor.from_llm(llm)

In [6]:
compression_retriver = ContextualCompressionRetriever(
	base_retriever=base_retriver,
	base_compressor=compressor
)

In [7]:
# Query the retriever
query = "What is photosynthesis?"

In [8]:
compression_results = compression_retriver.invoke(query)

In [9]:
for i, doc in enumerate(compression_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)



--- Result 1 ---
Photosynthesis is the process by which green plants convert sunlight into energy.

--- Result 2 ---
Photosynthesis does not occur in animal cells.

--- Result 3 ---
The chlorophyll in plant cells captures sunlight during photosynthesis.
